In [9]:
from math import pi

import pandas as pd
import numpy as np

from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.palettes import Spectral6
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.core.properties import value

In [2]:
import numpy as np

from bokeh.layouts import grid, column
from bokeh.models import CustomJS, Slider, ColumnDataSource
from bokeh.plotting import figure, output_file, show

output_file('dashboard.html')

tools = 'pan'


def bollinger():
    # Define Bollinger Bands.
    upperband = np.random.randint(100, 150+1, size=100)
    lowerband = upperband - 100
    x_data = np.arange(1, 101)

    # Bollinger shading glyph:
    band_x = np.append(x_data, x_data[::-1])
    band_y = np.append(lowerband, upperband[::-1])

    p = figure(x_axis_type='datetime', tools=tools)
    p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)

    p.title.text = 'Bollinger Bands'
    p.title_location = 'left'
    p.title.align = 'left'
    p.plot_height = 600
    p.plot_width = 800
    p.grid.grid_line_alpha = 0.4
    return [p]


def slider():
    x = np.linspace(0, 10, 100)
    y = np.sin(x)

    source = ColumnDataSource(data=dict(x=x, y=y))

    plot = figure(
        y_range=(-10, 10), tools='', toolbar_location=None,
        title="Sliders example")
    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

    callback = CustomJS(args=dict(source=source), code="""
        var data = source.data;
        var A = amp.value;
        var k = freq.value;
        var phi = phase.value;
        var B = offset.value;
        var x = data['x']
        var y = data['y']
        for (var i = 0; i < x.length; i++) {
            y[i] = B + A*Math.sin(k*x[i]+phi);
        }
        source.change.emit();
    """)

    amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude", callback=callback, callback_policy='mouseup')
    callback.args["amp"] = amp_slider

    freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency", callback=callback)
    callback.args["freq"] = freq_slider

    phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title="Phase", callback=callback)
    callback.args["phase"] = phase_slider

    offset_slider = Slider(start=-5, end=5, value=0, step=.1, title="Offset", callback=callback)
    callback.args["offset"] = offset_slider

    widgets = column(amp_slider, freq_slider, phase_slider, offset_slider)
    return [widgets, plot]


def linked_panning():
    N = 100
    x = np.linspace(0, 4 * np.pi, N)
    y1 = np.sin(x)
    y2 = np.cos(x)
    y3 = np.sin(x) + np.cos(x)

    s1 = figure(tools=tools)
    s1.circle(x, y1, color="navy", size=8, alpha=0.5)
    s2 = figure(tools=tools, x_range=s1.x_range, y_range=s1.y_range)
    s2.circle(x, y2, color="firebrick", size=8, alpha=0.5)
    s3 = figure(tools='pan, box_select', x_range=s1.x_range)
    s3.circle(x, y3, color="olive", size=8, alpha=0.5)
    return [s1, s2, s3]

l = grid([
    bollinger(),
    slider(),
    linked_panning(),
        ], sizing_mode='stretch_both')

show(l)

In [8]:
import numpy as np

from bokeh.layouts import grid, column
from bokeh.models import CustomJS, Slider, ColumnDataSource
from bokeh.plotting import figure, output_file, show

output_file('dashboard.html')

tools = 'pan'


def linked_panning():
    N = 100
    x = np.linspace(0, 4 * np.pi, N)
    y1 = np.sin(x)
    y2 = np.cos(x)
    y3 = np.sin(x) + np.cos(x)

    s1 = figure(tools=tools)
    s1.circle(x, y1, color="navy", size=8, alpha=0.5)
   # s2 = figure(tools=tools, x_range=s1.x_range, y_range=s1.y_range)
   # s2.circle(x, y2, color="firebrick", size=8, alpha=0.5)
    s3 = figure(tools='pan, box_select', x_range=s1.x_range)
    s3.circle(x, y3, color="olive", size=8, alpha=0.5)
    #return [s1, s2, s3]
    return [s1, s3]

l = grid([
        linked_panning(),
        ], #sizing_mode='stretch_both'
        )

show(l)

In [22]:
import numpy as np

from bokeh.layouts import grid, column
from bokeh.models import CustomJS, Slider, ColumnDataSource
from bokeh.plotting import figure, output_file, show

output_file('dashboard.html')

tools = 'pan'

df2 = pd.read_excel("BCI Portal Request.xlsx", sheet_name = 'FY19')
df3 = df2.drop_duplicates(subset='PID', keep='first')

###########################################################################################################################

#####"BCS - BCI BOOKINGS "###############################3

def bcs():
        df = pd.read_excel("FY19_BCS Bookings.xlsx", skiprows = [0, 1, 2, 3], sheet_name = 'BCI Bookings')
        df1 = df.drop_duplicates(subset='Project End Customer Name', keep='first')


        s = pd.value_counts(df1["SALES_NODE_LEVEL_1"].values, sort=False)

        regions = []
        for i in s.index:
            regions.append(i)


        regions_values = []
        for i in s.values:
            regions_values.append(i)

        x1 = {regions[0]:regions_values[0], regions[1]:regions_values[1], regions[2]:regions_values[2]}


        data = pd.Series(x1).reset_index(name='value').rename(columns={'index':'region'})
        data['angle'] = data['value']/data['value'].sum() * 2*pi
        data['color'] = Category20c[len(x1)]

        p = figure(plot_height=350, title="BCS - BCI BOOKINGS ", toolbar_location=None,
                   tools="hover", tooltips="@region: @value", x_range=(-0.5, 1.0))

        p.wedge(x=0, y=1, radius=0.4,
                start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                line_color="white", fill_color='color', legend='region', source=data)


        p.axis.axis_label=None
        p.axis.visible=False
        p.grid.grid_line_color = None

        return p


###########################################################################################################################

def bcs1():
    df2 = pd.read_excel("BCI Portal Request.xlsx", sheet_name = 'FY19')
    df3 = df2.drop_duplicates(subset='PID', keep='first')
    s2 = pd.value_counts(df3["CUSTOMER"].values, sort=False)

    customer = []
    for i in s2.index:
        customer.append(i)

    customer_values = []
    for i in s2.values:
        customer_values.append(i)

    source = ColumnDataSource(data=dict(labels=customer, values=customer_values))

    p = figure(x_range=customer, plot_height=450, plot_width = 900,toolbar_location=None, title="Region Information",
    tools="hover", tooltips="@labels: @values")

    p.vbar(x='labels', top='values', width=0.6, source=source, legend="labels",
    line_color='white', fill_color=factor_cmap('labels', palette=Spectral6, factors=customer))

    p.xgrid.grid_line_color = None
    p.y_range.start = min(customer_values) 
    p.y_range.end = max(customer_values) + 30
    p.legend.orientation = "horizontal"
    p.legend.orientation = "vertical"
    p.legend.location = "top_right"

    return p


###########################################################################################################################

def bcs2():
    
    customer_data1 = df3.groupby(['CUSTOMER','CAPTURED IN BCI'])
    total_values = customer_data1.count()
    fruits = ['Brownfield', 'Greenfield']
    years = ["Yes", "No"]
    colors = ["#008000", "#ff0000"]

    data = {'fruits' : fruits,
            'Yes'   : [total_values.loc["Brownfield", "Yes"]["PID"], total_values.loc["Greenfield", "Yes"]["PID"]],
            'No'   : [total_values.loc["Brownfield", "No"]["PID"], total_values.loc["Greenfield", "No"]["PID"]]
            }

    p = figure(x_range=fruits, plot_height=250, title="BCI CUSTOMER STATUS",
               toolbar_location=None, tools="hover", tooltips="$name @fruits: @$name")

    p.vbar_stack(years, x='fruits', width=0.9, color=colors, source=data,
                 legend=[value(x) for x in years])

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.axis.minor_tick_line_color = None
    p.outline_line_color = None
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    
    return p

###########################################################################################################################

l = grid([
        bcs(),
        bcs1(),
        bcs2()], #sizing_mode='stretch_both'
        )

show(l)